In [3]:
import joblib
from pandas import read_csv
import pandas as pd
from catboost import CatBoostClassifier

ModuleNotFoundError: No module named 'sklearn'

In [2]:
test = read_csv("NTI_data/test.csv")
train_target = read_csv("NTI_data/train_target.csv")
transactions_test = read_csv("NTI_data/transactions_test.csv")
transactions_train = read_csv("NTI_data/transactions_train.csv")

In [3]:
#Train
#Price features
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','max']).reset_index()

#Category count
amount_of_purchase=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()
counts_train=amount_of_purchase.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
counts_train=counts_train.fillna(0)
counts_train.columns=['small_group_' + str(i) for i in counts_train.columns]

#Category sum
sum_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].sum()
sum_train=sum_of_group.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
sum_train=sum_train.fillna(0)
sum_train.columns=['small_group_sum_'+str(i) for i in sum_train.columns]

#Category min and max
min_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
min_train=sum_of_group.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
min_train=min_train.fillna(0)
min_train.columns=['small_group_min_'+str(i) for i in min_train.columns]

max_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
max_train=sum_of_group.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
max_train=max_train.fillna(0)
max_train.columns=['small_group_max_'+str(i) for i in max_train.columns]

#Category average sum
avg_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].mean()
avg_train=avg_of_group.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
avg_train=avg_train.fillna(0)
avg_train.columns=['small_group_avg_'+str(i) for i in avg_train.columns]

#Merge all the features
counts_train=pd.merge(sum_train,counts_train,on='client_id')
counts_train=pd.merge(avg_train,counts_train,on='client_id')
counts_train=pd.merge(min_train,counts_train,on='client_id')
counts_train=pd.merge(max_train,counts_train,on='client_id')
train=pd.merge(train_target,agg_features,on='client_id')
train=pd.merge(train,counts_train.reset_index(),on='client_id')

In [13]:
train_size = 500
train_data = train.head(train_size)
test_data = train.tail(29000)

X_train = train_data.drop('bins', 1)
y_train = train_data['bins']

X_test = test_data.drop('bins', 1)
y_test = test_data['bins']

In [14]:
cat = CatBoostClassifier(iterations = 2000, learning_rate = 0.04, depth = 5, loss_function = 'MultiClass', eval_metric='Accuracy', random_state = 53)
cat.fit(X_train, y_train)

0:	learn: 0.5040000	total: 38.6ms	remaining: 1m 17s
1:	learn: 0.5320000	total: 69.2ms	remaining: 1m 9s
2:	learn: 0.5920000	total: 101ms	remaining: 1m 7s
3:	learn: 0.6160000	total: 132ms	remaining: 1m 5s
4:	learn: 0.6680000	total: 166ms	remaining: 1m 6s
5:	learn: 0.6760000	total: 212ms	remaining: 1m 10s
6:	learn: 0.6760000	total: 245ms	remaining: 1m 9s
7:	learn: 0.6600000	total: 281ms	remaining: 1m 10s
8:	learn: 0.6880000	total: 318ms	remaining: 1m 10s
9:	learn: 0.7040000	total: 354ms	remaining: 1m 10s
10:	learn: 0.7040000	total: 389ms	remaining: 1m 10s
11:	learn: 0.7120000	total: 422ms	remaining: 1m 9s
12:	learn: 0.7400000	total: 459ms	remaining: 1m 10s
13:	learn: 0.7520000	total: 491ms	remaining: 1m 9s
14:	learn: 0.7440000	total: 523ms	remaining: 1m 9s
15:	learn: 0.7440000	total: 555ms	remaining: 1m 8s
16:	learn: 0.7480000	total: 586ms	remaining: 1m 8s
17:	learn: 0.7600000	total: 616ms	remaining: 1m 7s
18:	learn: 0.7640000	total: 653ms	remaining: 1m 8s
19:	learn: 0.7680000	total: 691m

160:	learn: 0.9400000	total: 5.36s	remaining: 1m 1s
161:	learn: 0.9400000	total: 5.39s	remaining: 1m 1s
162:	learn: 0.9440000	total: 5.42s	remaining: 1m 1s
163:	learn: 0.9480000	total: 5.45s	remaining: 1m 1s
164:	learn: 0.9400000	total: 5.48s	remaining: 1m
165:	learn: 0.9440000	total: 5.51s	remaining: 1m
166:	learn: 0.9440000	total: 5.54s	remaining: 1m
167:	learn: 0.9440000	total: 5.58s	remaining: 1m
168:	learn: 0.9520000	total: 5.61s	remaining: 1m
169:	learn: 0.9520000	total: 5.64s	remaining: 1m
170:	learn: 0.9520000	total: 5.67s	remaining: 1m
171:	learn: 0.9520000	total: 5.7s	remaining: 1m
172:	learn: 0.9520000	total: 5.73s	remaining: 1m
173:	learn: 0.9520000	total: 5.76s	remaining: 1m
174:	learn: 0.9440000	total: 5.8s	remaining: 1m
175:	learn: 0.9440000	total: 5.83s	remaining: 1m
176:	learn: 0.9480000	total: 5.86s	remaining: 1m
177:	learn: 0.9440000	total: 5.89s	remaining: 1m
178:	learn: 0.9480000	total: 5.93s	remaining: 1m
179:	learn: 0.9440000	total: 5.96s	remaining: 1m
180:	learn

322:	learn: 1.0000000	total: 10.6s	remaining: 54.8s
323:	learn: 1.0000000	total: 10.6s	remaining: 54.8s
324:	learn: 1.0000000	total: 10.6s	remaining: 54.8s
325:	learn: 1.0000000	total: 10.7s	remaining: 54.7s
326:	learn: 1.0000000	total: 10.7s	remaining: 54.7s
327:	learn: 1.0000000	total: 10.7s	remaining: 54.6s
328:	learn: 1.0000000	total: 10.7s	remaining: 54.6s
329:	learn: 1.0000000	total: 10.8s	remaining: 54.5s
330:	learn: 1.0000000	total: 10.8s	remaining: 54.5s
331:	learn: 1.0000000	total: 10.8s	remaining: 54.4s
332:	learn: 1.0000000	total: 10.9s	remaining: 54.4s
333:	learn: 1.0000000	total: 10.9s	remaining: 54.4s
334:	learn: 1.0000000	total: 10.9s	remaining: 54.3s
335:	learn: 1.0000000	total: 11s	remaining: 54.3s
336:	learn: 1.0000000	total: 11s	remaining: 54.2s
337:	learn: 1.0000000	total: 11s	remaining: 54.2s
338:	learn: 1.0000000	total: 11s	remaining: 54.1s
339:	learn: 1.0000000	total: 11.1s	remaining: 54.1s
340:	learn: 1.0000000	total: 11.1s	remaining: 54s
341:	learn: 1.0000000	

484:	learn: 1.0000000	total: 15.8s	remaining: 49.3s
485:	learn: 1.0000000	total: 15.8s	remaining: 49.2s
486:	learn: 1.0000000	total: 15.8s	remaining: 49.2s
487:	learn: 1.0000000	total: 15.9s	remaining: 49.2s
488:	learn: 1.0000000	total: 15.9s	remaining: 49.1s
489:	learn: 1.0000000	total: 15.9s	remaining: 49.1s
490:	learn: 1.0000000	total: 16s	remaining: 49.1s
491:	learn: 1.0000000	total: 16s	remaining: 49s
492:	learn: 1.0000000	total: 16s	remaining: 49s
493:	learn: 1.0000000	total: 16.1s	remaining: 48.9s
494:	learn: 1.0000000	total: 16.1s	remaining: 48.9s
495:	learn: 1.0000000	total: 16.1s	remaining: 48.9s
496:	learn: 1.0000000	total: 16.1s	remaining: 48.8s
497:	learn: 1.0000000	total: 16.2s	remaining: 48.8s
498:	learn: 1.0000000	total: 16.2s	remaining: 48.8s
499:	learn: 1.0000000	total: 16.2s	remaining: 48.7s
500:	learn: 1.0000000	total: 16.3s	remaining: 48.7s
501:	learn: 1.0000000	total: 16.3s	remaining: 48.7s
502:	learn: 1.0000000	total: 16.3s	remaining: 48.6s
503:	learn: 1.0000000	

645:	learn: 1.0000000	total: 21s	remaining: 44s
646:	learn: 1.0000000	total: 21s	remaining: 44s
647:	learn: 1.0000000	total: 21.1s	remaining: 44s
648:	learn: 1.0000000	total: 21.1s	remaining: 43.9s
649:	learn: 1.0000000	total: 21.1s	remaining: 43.9s
650:	learn: 1.0000000	total: 21.2s	remaining: 43.9s
651:	learn: 1.0000000	total: 21.2s	remaining: 43.8s
652:	learn: 1.0000000	total: 21.2s	remaining: 43.8s
653:	learn: 1.0000000	total: 21.3s	remaining: 43.8s
654:	learn: 1.0000000	total: 21.3s	remaining: 43.8s
655:	learn: 1.0000000	total: 21.3s	remaining: 43.7s
656:	learn: 1.0000000	total: 21.4s	remaining: 43.7s
657:	learn: 1.0000000	total: 21.4s	remaining: 43.7s
658:	learn: 1.0000000	total: 21.4s	remaining: 43.6s
659:	learn: 1.0000000	total: 21.5s	remaining: 43.6s
660:	learn: 1.0000000	total: 21.5s	remaining: 43.6s
661:	learn: 1.0000000	total: 21.5s	remaining: 43.6s
662:	learn: 1.0000000	total: 21.6s	remaining: 43.5s
663:	learn: 1.0000000	total: 21.6s	remaining: 43.5s
664:	learn: 1.0000000	

...

1286:	learn: 1.0000000	total: 42s	remaining: 23.3s
1287:	learn: 1.0000000	total: 42s	remaining: 23.2s
1288:	learn: 1.0000000	total: 42.1s	remaining: 23.2s
1289:	learn: 1.0000000	total: 42.1s	remaining: 23.2s
1290:	learn: 1.0000000	total: 42.1s	remaining: 23.1s
1291:	learn: 1.0000000	total: 42.2s	remaining: 23.1s
1292:	learn: 1.0000000	total: 42.2s	remaining: 23.1s
1293:	learn: 1.0000000	total: 42.2s	remaining: 23s
1294:	learn: 1.0000000	total: 42.3s	remaining: 23s
1295:	learn: 1.0000000	total: 42.3s	remaining: 23s
1296:	learn: 1.0000000	total: 42.3s	remaining: 22.9s
1297:	learn: 1.0000000	total: 42.3s	remaining: 22.9s
1298:	learn: 1.0000000	total: 42.4s	remaining: 22.9s
1299:	learn: 1.0000000	total: 42.4s	remaining: 22.8s
1300:	learn: 1.0000000	total: 42.4s	remaining: 22.8s
1301:	learn: 1.0000000	total: 42.5s	remaining: 22.8s
1302:	learn: 1.0000000	total: 42.5s	remaining: 22.7s
1303:	learn: 1.0000000	total: 42.5s	remaining: 22.7s
1304:	learn: 1.0000000	total: 42.6s	remaining: 22.7s
130

1443:	learn: 1.0000000	total: 47.2s	remaining: 18.2s
1444:	learn: 1.0000000	total: 47.2s	remaining: 18.1s
1445:	learn: 1.0000000	total: 47.3s	remaining: 18.1s
1446:	learn: 1.0000000	total: 47.3s	remaining: 18.1s
1447:	learn: 1.0000000	total: 47.3s	remaining: 18s
1448:	learn: 1.0000000	total: 47.4s	remaining: 18s
1449:	learn: 1.0000000	total: 47.4s	remaining: 18s
1450:	learn: 1.0000000	total: 47.4s	remaining: 17.9s
1451:	learn: 1.0000000	total: 47.4s	remaining: 17.9s
1452:	learn: 1.0000000	total: 47.5s	remaining: 17.9s
1453:	learn: 1.0000000	total: 47.5s	remaining: 17.8s
1454:	learn: 1.0000000	total: 47.5s	remaining: 17.8s
1455:	learn: 1.0000000	total: 47.6s	remaining: 17.8s
1456:	learn: 1.0000000	total: 47.6s	remaining: 17.7s
1457:	learn: 1.0000000	total: 47.6s	remaining: 17.7s
1458:	learn: 1.0000000	total: 47.7s	remaining: 17.7s
1459:	learn: 1.0000000	total: 47.7s	remaining: 17.6s
1460:	learn: 1.0000000	total: 47.7s	remaining: 17.6s
1461:	learn: 1.0000000	total: 47.8s	remaining: 17.6s

1603:	learn: 1.0000000	total: 52.7s	remaining: 13s
1604:	learn: 1.0000000	total: 52.7s	remaining: 13s
1605:	learn: 1.0000000	total: 52.7s	remaining: 12.9s
1606:	learn: 1.0000000	total: 52.8s	remaining: 12.9s
1607:	learn: 1.0000000	total: 52.8s	remaining: 12.9s
1608:	learn: 1.0000000	total: 52.8s	remaining: 12.8s
1609:	learn: 1.0000000	total: 52.9s	remaining: 12.8s
1610:	learn: 1.0000000	total: 52.9s	remaining: 12.8s
1611:	learn: 1.0000000	total: 52.9s	remaining: 12.7s
1612:	learn: 1.0000000	total: 53s	remaining: 12.7s
1613:	learn: 1.0000000	total: 53s	remaining: 12.7s
1614:	learn: 1.0000000	total: 53s	remaining: 12.6s
1615:	learn: 1.0000000	total: 53.1s	remaining: 12.6s
1616:	learn: 1.0000000	total: 53.1s	remaining: 12.6s
1617:	learn: 1.0000000	total: 53.1s	remaining: 12.5s
1618:	learn: 1.0000000	total: 53.2s	remaining: 12.5s
1619:	learn: 1.0000000	total: 53.2s	remaining: 12.5s
1620:	learn: 1.0000000	total: 53.2s	remaining: 12.4s
1621:	learn: 1.0000000	total: 53.3s	remaining: 12.4s
162

1763:	learn: 1.0000000	total: 58s	remaining: 7.76s
1764:	learn: 1.0000000	total: 58s	remaining: 7.72s
1765:	learn: 1.0000000	total: 58s	remaining: 7.69s
1766:	learn: 1.0000000	total: 58.1s	remaining: 7.66s
1767:	learn: 1.0000000	total: 58.1s	remaining: 7.63s
1768:	learn: 1.0000000	total: 58.1s	remaining: 7.59s
1769:	learn: 1.0000000	total: 58.2s	remaining: 7.56s
1770:	learn: 1.0000000	total: 58.2s	remaining: 7.53s
1771:	learn: 1.0000000	total: 58.2s	remaining: 7.49s
1772:	learn: 1.0000000	total: 58.3s	remaining: 7.46s
1773:	learn: 1.0000000	total: 58.3s	remaining: 7.43s
1774:	learn: 1.0000000	total: 58.3s	remaining: 7.4s
1775:	learn: 1.0000000	total: 58.4s	remaining: 7.36s
1776:	learn: 1.0000000	total: 58.4s	remaining: 7.33s
1777:	learn: 1.0000000	total: 58.5s	remaining: 7.3s
1778:	learn: 1.0000000	total: 58.5s	remaining: 7.26s
1779:	learn: 1.0000000	total: 58.5s	remaining: 7.23s
1780:	learn: 1.0000000	total: 58.5s	remaining: 7.2s
1781:	learn: 1.0000000	total: 58.6s	remaining: 7.17s
17

1925:	learn: 1.0000000	total: 1m 3s	remaining: 2.43s
1926:	learn: 1.0000000	total: 1m 3s	remaining: 2.4s
1927:	learn: 1.0000000	total: 1m 3s	remaining: 2.37s
1928:	learn: 1.0000000	total: 1m 3s	remaining: 2.33s
1929:	learn: 1.0000000	total: 1m 3s	remaining: 2.3s
1930:	learn: 1.0000000	total: 1m 3s	remaining: 2.27s
1931:	learn: 1.0000000	total: 1m 3s	remaining: 2.23s
1932:	learn: 1.0000000	total: 1m 3s	remaining: 2.2s
1933:	learn: 1.0000000	total: 1m 3s	remaining: 2.17s
1934:	learn: 1.0000000	total: 1m 3s	remaining: 2.14s
1935:	learn: 1.0000000	total: 1m 3s	remaining: 2.1s
1936:	learn: 1.0000000	total: 1m 3s	remaining: 2.07s
1937:	learn: 1.0000000	total: 1m 3s	remaining: 2.04s
1938:	learn: 1.0000000	total: 1m 3s	remaining: 2s
1939:	learn: 1.0000000	total: 1m 3s	remaining: 1.97s
1940:	learn: 1.0000000	total: 1m 3s	remaining: 1.94s
1941:	learn: 1.0000000	total: 1m 3s	remaining: 1.91s
1942:	learn: 1.0000000	total: 1m 3s	remaining: 1.87s
1943:	learn: 1.0000000	total: 1m 3s	remaining: 1.84s


In [15]:
cat.score(X_test, y_test)

0.5359655172413793